In [1]:
import pandas as pd
import numpy as np

In [2]:
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = 'plotly_white'

# 5 MeV

In [3]:
test_targets = pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/ideal/test/targets/e+_5/targets_test_0.csv')
for i in range(99):
    test_targets = test_targets.append(
        pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/ideal/test/targets/e+_5/targets_test_{}.csv'.format(i+1))
    )
test_targets = test_targets.reset_index().drop('index', axis=1)

In [4]:
test_targets['edepR'] = (test_targets['edepX']**2 + test_targets['edepY']**2 + test_targets['edepZ']**2)**0.5

In [5]:
test_targets = test_targets[test_targets['edepR'] < 17.2]

In [6]:
einit = pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/ideal/test/targets/e+_5/einit_test_0.csv')
for i in range(99):
    einit = einit.append(
        pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/ideal/test/targets/e+_5/einit_test_{}.csv'.format(i+1))
    )
einit = einit.reset_index().drop('index', axis=1)
einit = einit.iloc[test_targets.index]

In [7]:
en_diff = test_targets['edep'] - einit['einit']

In [8]:
# fig = go.Figure()

# fig.add_trace(
#     go.Histogram(
#         x=en_diff,
#         xbins=dict(
#             size=0.1
#         ),
#         histnorm='probability',
#     )
# )

# fig.update_yaxes(type="log")

# fig.show()

In [9]:
from tqdm import tqdm

train_targets = pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/real/train/targets/targets_train_0.csv')
for i in tqdm(range(1002)):
    train_targets = train_targets.append(
        pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/real/train/targets/targets_train_{}.csv'.format(i+1))
    )
train_targets = train_targets.reset_index().drop('index', axis=1)

100%|██████████| 1002/1002 [02:03<00:00,  8.14it/s]


In [10]:
train_targets['edepR'] = (train_targets['edepX']**2 + train_targets['edepY']**2 + train_targets['edepZ']**2)**0.5

In [11]:
train_targets = train_targets[train_targets['edepR'] < 17.2]

In [12]:
einit = pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/real/train/targets/einit_train_0.csv')
for i in tqdm(range(1002)):
    einit = einit.append(
        pd.read_csv('/mnt/cephfs/ml_data/mc_2021/raw_data/real/train/targets/einit_train_{}.csv'.format(i+1))
    )
einit = einit.reset_index().drop('index', axis=1)
einit = einit.iloc[train_targets.index]

100%|██████████| 1002/1002 [01:07<00:00, 14.87it/s]


In [13]:
en_diff = train_targets['edep'] - einit['einit']

In [14]:
# fig = go.Figure()

# fig.add_trace(
#     go.Histogram(
#         x=en_diff,
#         xbins=dict(
#             size=0.2
#         ),
#         histnorm='probability',
#     )
# )

# fig.update_yaxes(type="log")

# fig.show()

In [16]:
y = pd.DataFrame(en_diff, columns=['targets'])
y['targets'] = np.zeros(shape=len(y), dtype=np.int32)
y.loc[en_diff[abs(en_diff - 0.511) > train_targets['edep']*0.03 / train_targets['edep']**0.5].index] = 1

In [17]:
y.to_csv('/mnt/cephfs/ml_data/mc_2021/processed_data/train_real_Einit_clf.csv', index=False)